In [3]:
#!/usr/bin/env python3
"""
29 – Reconciliation Reporting (Audit + Ongoing Diagnostics)

Reads reconciliation outputs and produces a PDF report:
- executed_trades.csv (append-only ledger)
- reconciliation_log.csv (diagnostic mirror; optional)
- broker_fills_manual.csv (optional; pending fill detection)
- live_portfolio.csv (state snapshot; drift check)

Slippage Reporting (REPORT BOTH):
- SigFill Slip: signal_price -> fill_price
- OrdFill Slip: order_price  -> fill_price

Assumptions for slippage dollars:
  BUY  : (fill - ref) * shares
  SELL : (ref - fill) * shares
Positive = cost (worse), negative = improvement (better)

Portfolio Reporting:
- Average cost per share (moving average, handles multiple buys/sells)
- Current price from your price database (parquet)
- Date first acquired (for current open position)
- Current return % vs avg cost
- Weight % using current price

NEW:
- Front-page YTD performance table comparing Strategy vs SPY (like your screenshot).
  SPY is sourced ONLY from 8-SPY_200DMA_regime.parquet (spy_close).
"""

# ============================================================
# Imports
# ============================================================

import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle,
    PageBreak, Image
)
from pandas.errors import EmptyDataError

# ============================================================
# Configuration
# ============================================================

LIVE_ROOT = "./27a-2G_live_trading"

LIVE_PORTFOLIO_FILE  = f"{LIVE_ROOT}/live_portfolio.csv"
MANUAL_FILLS_FILE    = f"{LIVE_ROOT}/broker_fills_manual.csv"
EXECUTED_TRADES_FILE = f"{LIVE_ROOT}/executed_trades.csv"
RECON_LOG_FILE       = f"{LIVE_ROOT}/reconciliation_log.csv"

# source for slope_rank
MASTER_TRADES_FILE   = f"{LIVE_ROOT}/master_trades.csv"
MASTER_RANKINGS_FILE = f"{LIVE_ROOT}/master_rankings.csv"

OUTPUT_DIR = "./29a-2G_reconciliation_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

MIN_YEAR_FOR_REPORT = 1999
MAX_YEAR_FOR_REPORT = 2025
TRADING_DAYS_PER_YEAR = 252

# ---- Price database (your maintained price history) ----
PRICE_DB_FILE = "./12-tradable_sp500_universe/12-tradable_sp500_universe.parquet"
PRICE_DATE_COL = "date"
PRICE_TICKER_COL = "ticker"
PRICE_PX_COL = "close_adj"

# ---- SPY regime file (FOR SPY PERFORMANCE TABLE) ----
SPY_REGIME_FILE_PRIMARY = r"C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\8-SPY_200DMA_market_regime\8-SPY_200DMA_regime.parquet"
SPY_REGIME_FILE_FALLBACK = "./8-SPY_200DMA_market_regime/8-SPY_200DMA_regime.parquet"

# ============================================================
# Slippage column names (define EARLY so helpers can reference)
# ============================================================

SIG_SLIP_COL = "slip_sigfill_dollars"   # signal_price -> fill_price
ORD_SLIP_COL = "slip_ordfill_dollars"   # order_price  -> fill_price

SIG_LABEL = "SigFill Slip (signal_price→fill_price)"
ORD_LABEL = "OrdFill Slip (order_price→fill_price)"

# ============================================================
# Helpers
# ============================================================

def _safe_read_csv(path: str) -> pd.DataFrame:
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        return pd.DataFrame()
    try:
        return pd.read_csv(path)
    except EmptyDataError:
        return pd.DataFrame()

def _to_dt(s) -> pd.Series:
    return pd.to_datetime(s, errors="coerce")

def _norm_side(s: pd.Series) -> pd.Series:
    return s.astype(str).str.upper().str.strip()

def _norm_ticker(s: pd.Series) -> pd.Series:
    return s.astype(str).str.upper().str.strip()

def _fmt(v, col=None):
    if v is None:
        return "N/A"
    if isinstance(v, (float, np.floating)) and np.isnan(v):
        return "N/A"
    if isinstance(v, (pd.Timestamp, np.datetime64)):
        return str(pd.Timestamp(v).date())
    if isinstance(v, (int, np.integer)):
        return str(int(v))
    if isinstance(v, (float, np.floating)):
        money_cols = {
            "fees","broker_fee","net_cash_impact","gross_notional","cash",
            "sig_slip_net_dollars","sig_slip_gross_cost_dollars","sig_slip_gross_improve_dollars",
            "ord_slip_net_dollars","ord_slip_gross_cost_dollars","ord_slip_gross_improve_dollars",
            "ytd_sig_slip_net_dollars","ytd_sig_slip_gross_cost_dollars","ytd_sig_slip_gross_improve_dollars",
            "ytd_ord_slip_net_dollars","ytd_ord_slip_gross_cost_dollars","ytd_ord_slip_gross_improve_dollars",
            "sig_slip_per_trade_dollars","ytd_sig_slip_per_trade_dollars",
            "ord_slip_per_trade_dollars","ytd_ord_slip_per_trade_dollars",
            SIG_SLIP_COL, ORD_SLIP_COL,
            "avg_cost","current_price","market_value",
                "target_value",           # ADD
            "current_value",          # ADD
        }
        bps_cols = {
            "sig_slip_net_bps","ord_slip_net_bps",
            "ytd_sig_slip_net_bps","ytd_ord_slip_net_bps",
        }
        pct_cols = {
            "weight_pct",
            "position_return_pct",
             "target_weight",          # ADD
            "current_weight",         # ADD
            "weight_change",          # ADD
            "capped_weight",          # ADD
            "raw_weight"              # ADD
            }

        if col in pct_cols:
            return f"{float(v):,.2f}%"
        if col in money_cols:
            return f"{float(v):,.2f}"
        if col in bps_cols:
            return f"{float(v):,.2f}"
        return f"{float(v):0.4f}"
    return str(v)

def _fmt_pct(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "N/A"
    return f"{x*100.0:,.2f}%"

def _fmt_num(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "N/A"
    return f"{x:,.2f}"

def make_table(story, title, df: pd.DataFrame, cols, font_size=8, bold_last_row=False):
    story.append(Paragraph(title, styles["Small"]))
    if df is None or df.empty:
        story.append(Paragraph("None", styles["Tiny"]))
        story.append(Spacer(1, 0.10 * inch))
        return

    cols = [c for c in cols if c in df.columns]
    data = [cols]
    for _, row in df[cols].iterrows():
        data.append([_fmt(row[c], c) for c in cols])

    tbl = Table(data, hAlign="LEFT", repeatRows=1)

    style_cmds = [
        ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
        ("GRID",(0,0),(-1,-1),0.25,colors.grey),
        ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
        ("FONTSIZE",(0,0),(-1,-1),font_size),
        ("ALIGN",(1,1),(-1,-1),"RIGHT"),
    ]

    if bold_last_row and len(data) > 1:
        last = len(data) - 1
        style_cmds += [
            ("LINEABOVE",(0,last),(-1,last),0.75,colors.black),
            ("BACKGROUND",(0,last),(-1,last),colors.whitesmoke),
            ("FONTNAME",(0,last),(-1,last),"Helvetica-Bold"),
        ]

    tbl.setStyle(TableStyle(style_cmds))
    story.append(tbl)
    story.append(Spacer(1, 0.12 * inch))

def append_totals_row(df: pd.DataFrame, sum_cols: list, label_col: str = "ticker", label: str = "TOTAL") -> pd.DataFrame:
    if df is None or df.empty:
        return df
    out = df.copy()
    totals = {c: np.nan for c in out.columns}
    if label_col in out.columns:
        totals[label_col] = label
    for c in sum_cols:
        if c in out.columns:
            totals[c] = pd.to_numeric(out[c], errors="coerce").sum(skipna=True)
    return pd.concat([out, pd.DataFrame([totals])], ignore_index=True)

def add_sequential_display_cash(
    df: pd.DataFrame,
    start_cash: float,
    *,
    side_col: str = "side",
    shares_col: str = "shares",
    fill_col: str = "fill_price",
    fee_col: str = "broker_fee",
) -> tuple[pd.DataFrame, float]:
    """
    Recomputes sequential cash_before/cash_after in *row order* of df.

    Cash impact rule:
      BUY  : cash -= shares*fill_price + fee
      SELL : cash += shares*fill_price - fee

    Returns (df_with_cols, ending_cash).
    """
    if df is None or df.empty:
        return df, float(start_cash)

    out = df.copy()

    # normalize inputs
    side = _norm_side(out[side_col])
    shares = pd.to_numeric(out.get(shares_col, 0), errors="coerce").fillna(0.0)
    fill = pd.to_numeric(out.get(fill_col, np.nan), errors="coerce")
    fee = pd.to_numeric(out.get(fee_col, 0.0), errors="coerce").fillna(0.0)

    cash = float(start_cash) if pd.notna(start_cash) else 0.0
    cb = []
    ca = []
    delta_list = []

    for i in range(len(out)):
        cb.append(cash)

        sh = float(shares.iloc[i])
        px = float(fill.iloc[i]) if pd.notna(fill.iloc[i]) else np.nan
        f  = float(fee.iloc[i]) if pd.notna(fee.iloc[i]) else 0.0

        # if we can't price the trade, don't move cash (but keep the sequence intact)
        if sh <= 0 or pd.isna(px) or px <= 0:
            delta = 0.0
        else:
            if side.iloc[i] == "BUY":
                delta = -(sh * px) - f
            elif side.iloc[i] == "SELL":
                delta = (sh * px) - f
            else:
                delta = 0.0

        cash = cash + delta

        delta_list.append(delta)
        ca.append(cash)

    out["cash_before_display"] = cb
    out["cash_after_display"] = ca
    out["cash_delta_display"] = delta_list

    return out, float(cash)

def reconstruct_positions_from_ledger(exec_df: pd.DataFrame) -> dict:
    pos = {}
    if exec_df is None or exec_df.empty:
        return pos

    df = exec_df.copy()
    df["side"] = _norm_side(df["side"])
    df["ticker"] = _norm_ticker(df["ticker"])
    df["shares"] = pd.to_numeric(df["shares"], errors="coerce").fillna(0).astype(int)

    if "exec_ts" in df.columns:
        df["exec_ts"] = _to_dt(df["exec_ts"])
        df = df.sort_values(["exec_ts", "broker_order_id"], kind="mergesort")
    else:
        df = df.sort_values(["exec_date", "broker_order_id"], kind="mergesort")

    for _, r in df.iterrows():
        t = r["ticker"]
        sh = int(r["shares"])
        side = r["side"]
        if not t or sh <= 0:
            continue
        if side == "BUY":
            pos[t] = pos.get(t, 0) + sh
        elif side == "SELL":
            pos[t] = pos.get(t, 0) - sh
            if pos.get(t, 0) <= 0:
                pos.pop(t, None)
    return pos

def read_live_portfolio_snapshot(path: str) -> tuple[float, dict]:
    port = _safe_read_csv(path)
    if port.empty or "cash" not in port.columns:
        return (np.nan, {})
    cash = float(port.iloc[0]["cash"])
    pos = {}
    if "ticker" in port.columns and "shares" in port.columns:
        for _, r in port.iterrows():
            t = str(r.get("ticker", "")).strip().upper()
            sh = int(pd.to_numeric(r.get("shares", 0), errors="coerce") or 0)
            if t and sh != 0:
                pos[t] = sh
    return cash, pos

def prev_trading_day(d: pd.Timestamp, cal: pd.DatetimeIndex) -> pd.Timestamp:
    d = pd.Timestamp(d).normalize()
    if cal is None or len(cal) == 0:
        return pd.NaT
    i = int(cal.searchsorted(d) - 1)  # strictly before d
    return cal[i] if i >= 0 else pd.NaT

def trading_day_on_or_before(d: pd.Timestamp, cal: pd.DatetimeIndex) -> pd.Timestamp:
    d = pd.Timestamp(d).normalize()
    if cal is None or len(cal) == 0:
        return pd.NaT
    i = int(cal.searchsorted(d, side="right") - 1)  # <= d
    return cal[i] if i >= 0 else pd.NaT

# ============================================================
# Slippage Breakdown (cost vs improvement)
# ============================================================

def slippage_breakdown(df: pd.DataFrame, slip_col: str, notional_col: str = "gross_notional") -> dict:
    if df is None or df.empty or slip_col not in df.columns:
        return {"net_dollars": np.nan, "gross_cost_dollars": np.nan, "gross_improve_dollars": np.nan, "net_bps": np.nan}

    slip = pd.to_numeric(df[slip_col], errors="coerce")
    pos = slip.where(slip > 0, 0.0)
    neg = slip.where(slip < 0, 0.0)

    net = float(slip.sum(skipna=True))
    gross_cost = float(pos.sum(skipna=True))
    gross_improve = float((-neg).sum(skipna=True))

    if notional_col in df.columns:
        notional = pd.to_numeric(df[notional_col], errors="coerce").abs()
        denom = float(notional.sum(skipna=True))
    else:
        denom = 0.0

    net_bps = (net / denom) * 1e4 if denom > 0 else np.nan

    return {
        "net_dollars": net,
        "gross_cost_dollars": gross_cost,
        "gross_improve_dollars": gross_improve,
        "net_bps": net_bps,
    }

# ============================================================
# Price DB: fast last-known lookup
# ============================================================

def fast_price_lookup(px_array, date_val):
    date_val = np.datetime64(pd.Timestamp(date_val), "ns")
    dates = px_array["date"]
    idx = np.searchsorted(dates, date_val, side="right") - 1
    if idx < 0:
        return np.nan
    return float(px_array["px"][idx])

def fast_price_lookup_with_date(px_array, date_val):
    """
    Returns (price, date_used) where date_used is the actual price date chosen (last <= date_val).
    """
    if px_array is None:
        return (np.nan, pd.NaT)

    date_val = np.datetime64(pd.Timestamp(date_val), "ns")
    dates = px_array["date"]
    idx = np.searchsorted(dates, date_val, side="right") - 1
    if idx < 0:
        return (np.nan, pd.NaT)

    px = float(px_array["px"][idx])
    dt_used = pd.Timestamp(dates[idx]).normalize()
    return (px, dt_used)


def build_portfolio_snapshot_from_ledger(exec_df_slice: pd.DataFrame, px_by_ticker: dict, value_date: pd.Timestamp) -> pd.DataFrame:
    """
    Builds holdings from the executed ledger slice (positions as of the end of that slice),
    and prices each ticker using the last close <= value_date.
    """
    basis = compute_cost_basis_from_ledger(exec_df_slice)

    rows = []
    for t, info in basis.items():
        sh = int(info.get("shares_ledger", 0))
        if sh <= 0:
            continue

        avg_cost = info.get("avg_cost", np.nan)
        first_acq = info.get("first_acquired", pd.NaT)

        px = np.nan
        px_dt = pd.NaT
        arr = px_by_ticker.get(t)
        if arr is not None and pd.notna(value_date):
            px, px_dt = fast_price_lookup_with_date(arr, value_date)

        mv = (px * sh) if pd.notna(px) else np.nan
        ret_pct = np.nan
        if pd.notna(avg_cost) and avg_cost > 0 and pd.notna(px):
            ret_pct = (px / avg_cost - 1.0) * 100.0

        rows.append({
            "ticker": t,
            "shares": sh,
            "avg_cost": avg_cost,
            "current_price": px,
            "price_date_used": px_dt,
            "position_return_pct": ret_pct,
            "first_acquired": first_acq,
            "market_value": mv,
        })

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    total_mv = float(df["market_value"].sum(skipna=True))
    df["weight_pct"] = np.where(total_mv > 0, (df["market_value"] / total_mv) * 100.0, np.nan)
    df = df.sort_values("weight_pct", ascending=False, na_position="last").reset_index(drop=True)
    return df

# ============================================================
# Average cost / first acquired from ledger (moving average)
# ============================================================

def compute_cost_basis_from_ledger(exec_df: pd.DataFrame) -> dict:
    state = {}
    if exec_df is None or exec_df.empty:
        return state

    df = exec_df.copy()
    df["ticker"] = _norm_ticker(df.get("ticker", ""))
    df["side"] = _norm_side(df.get("side", ""))
    df["shares"] = pd.to_numeric(df.get("shares", 0), errors="coerce").fillna(0).astype(int)
    df["fill_price"] = pd.to_numeric(df.get("fill_price", np.nan), errors="coerce")
    df["broker_fee"] = pd.to_numeric(df.get("broker_fee", 0.0), errors="coerce").fillna(0.0)
    df["exec_date"] = _to_dt(df.get("exec_date"))

    if "exec_ts" in df.columns and df["exec_ts"].notna().any():
        df["exec_ts"] = _to_dt(df["exec_ts"])
        df = df.sort_values(["exec_ts", "broker_order_id"], kind="mergesort")
    else:
        df = df.sort_values(["exec_date", "broker_order_id"], kind="mergesort")

    for _, r in df.iterrows():
        t = r["ticker"]
        if not t:
            continue
        side = r["side"]
        sh = int(r["shares"])
        if sh <= 0:
            continue

        px = r["fill_price"]
        fee = float(r["broker_fee"] if pd.notna(r["broker_fee"]) else 0.0)
        dt = r["exec_date"]

        if t not in state:
            state[t] = {"shares": 0, "total_cost": 0.0, "first_acquired": pd.NaT}

        s = state[t]
        shares_before = int(s["shares"])
        total_cost_before = float(s["total_cost"])
        avg_cost_before = (total_cost_before / shares_before) if shares_before > 0 else np.nan

        if side == "BUY":
            if pd.isna(px) or px <= 0:
                continue
            if shares_before == 0 and pd.notna(dt):
                s["first_acquired"] = dt.normalize()
            s["shares"] = shares_before + sh
            s["total_cost"] = total_cost_before + (sh * float(px)) + fee

        elif side == "SELL":
            if shares_before <= 0:
                continue
            sell_sh = min(sh, shares_before)
            if not np.isnan(avg_cost_before):
                s["total_cost"] = max(0.0, total_cost_before - (avg_cost_before * sell_sh))
            s["shares"] = shares_before - sell_sh
            if s["shares"] <= 0:
                s["shares"] = 0
                s["total_cost"] = 0.0
                s["first_acquired"] = pd.NaT

    out = {}
    for t, s in state.items():
        sh = int(s["shares"])
        avg_cost = (float(s["total_cost"]) / sh) if sh > 0 else np.nan
        out[t] = {"shares_ledger": sh, "avg_cost": avg_cost, "first_acquired": s["first_acquired"]}
    return out

# ============================================================
# Attach slope_rank into executed ledger (from master_trades.csv)
# ============================================================

def attach_slope_rank(exec_df: pd.DataFrame) -> pd.DataFrame:
    mt = _safe_read_csv(MASTER_TRADES_FILE)
    if mt is None or mt.empty:
        return exec_df

    if "ticker" in mt.columns:
        mt["ticker"] = _norm_ticker(mt["ticker"])
    if "side" in mt.columns:
        mt["side"] = _norm_side(mt["side"])
    for c in ("signal_date", "exec_date"):
        if c in mt.columns:
            mt[c] = _to_dt(mt[c]).dt.normalize()

    if "slope_rank" in mt.columns:
        mt["slope_rank"] = pd.to_numeric(mt["slope_rank"], errors="coerce")
    else:
        return exec_df

    out = exec_df.copy()
    if "ticker" in out.columns:
        out["ticker"] = _norm_ticker(out["ticker"])
    if "side" in out.columns:
        out["side"] = _norm_side(out["side"])
    for c in ("signal_date", "exec_date"):
        if c in out.columns:
            out[c] = _to_dt(out[c]).dt.normalize()

    if "plan_id" in out.columns and "plan_id" in mt.columns:
        mt1 = mt[["plan_id", "slope_rank"]].drop_duplicates(subset=["plan_id"], keep="last")
        return out.merge(mt1, on="plan_id", how="left")

    keys = [k for k in ["signal_date", "exec_date", "ticker", "side"] if k in out.columns and k in mt.columns]
    if len(keys) == 4:
        mt2 = mt[keys + ["slope_rank"]].drop_duplicates(subset=keys, keep="last")
        return out.merge(mt2, on=keys, how="left")

    return out

# ============================================================
# Sort trades by slope-rank (best rank -> worst), buys then sells
# ============================================================

RANK_COL_CANDIDATES = ["slope_rank", "slope_rank_within_top", "rank"]

def sort_trades_by_rank(day_df: pd.DataFrame) -> tuple[pd.DataFrame, str | None]:
    if day_df is None or day_df.empty:
        return day_df, None

    rank_col = next((c for c in RANK_COL_CANDIDATES if c in day_df.columns), None)
    if rank_col is None:
        return day_df, None

    out = day_df.copy()
    out[rank_col] = pd.to_numeric(out[rank_col], errors="coerce")

    buys  = out[_norm_side(out["side"]) == "BUY"].copy()
    sells = out[_norm_side(out["side"]) == "SELL"].copy()

    buys  = buys.sort_values(rank_col, ascending=True, na_position="last", kind="mergesort")
    sells = sells.sort_values(rank_col, ascending=True, na_position="last", kind="mergesort")

    return pd.concat([buys, sells], ignore_index=True), rank_col

# ============================================================
# NEW: SPY series loader (from regime parquet)
# ============================================================

def load_spy_close_series() -> pd.Series:
    path = SPY_REGIME_FILE_PRIMARY if os.path.exists(SPY_REGIME_FILE_PRIMARY) else SPY_REGIME_FILE_FALLBACK
    if not os.path.exists(path):
        return pd.Series(dtype=float)

    df = pd.read_parquet(path)
    # Your file shows index named "Date"
    if "spy_close" not in df.columns:
        return pd.Series(dtype=float)

    if df.index.name is None or str(df.index.name).lower() not in ("date", "datetime"):
        # if Date is a column, try using it
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
            df = df.dropna(subset=["Date"]).set_index("Date")
        else:
            # last resort: coerce index to datetime
            df.index = pd.to_datetime(df.index, errors="coerce")

    df = df.sort_index()
    s = pd.to_numeric(df["spy_close"], errors="coerce").dropna()
    s.index = pd.to_datetime(s.index).normalize()
    return s

# ============================================================
# NEW: Strategy equity curve + performance metrics
# ============================================================

def build_strategy_equity_curve(exec_df: pd.DataFrame, px_by_ticker: dict, date_index: pd.DatetimeIndex) -> pd.Series:
    """
    Builds end-of-day equity = cash + sum(shares * close_adj) using:
      - cash simulated from fills (fill_price, broker_fee)
      - positions updated from BUY/SELL shares
      - marks to market using px_by_ticker fast lookup
    """
    if exec_df is None or exec_df.empty or len(date_index) == 0:
        return pd.Series(dtype=float)

    df = exec_df.copy()
    df["exec_ts"] = _to_dt(df.get("exec_ts"))
    df["exec_date"] = _to_dt(df.get("exec_date")).dt.normalize()
    df["ticker"] = _norm_ticker(df.get("ticker", ""))
    df["side"] = _norm_side(df.get("side", ""))
    df["shares"] = pd.to_numeric(df.get("shares", 0), errors="coerce").fillna(0).astype(int)
    df["fill_price"] = pd.to_numeric(df.get("fill_price", np.nan), errors="coerce")
    df["broker_fee"] = pd.to_numeric(df.get("broker_fee", 0.0), errors="coerce").fillna(0.0)

    # sort trades
    if df["exec_ts"].notna().any():
        df = df.sort_values(["exec_ts", "broker_order_id"], kind="mergesort")
    else:
        df = df.sort_values(["exec_date", "broker_order_id"], kind="mergesort")

    # start cash: first cash_before if present, else first cash_after, else 0
    cash0 = np.nan
    if "cash_before" in df.columns and df["cash_before"].notna().any():
        cash0 = float(pd.to_numeric(df["cash_before"], errors="coerce").dropna().iloc[0])
    elif "cash_after" in df.columns and df["cash_after"].notna().any():
        cash0 = float(pd.to_numeric(df["cash_after"], errors="coerce").dropna().iloc[0])
    if np.isnan(cash0):
        cash0 = 0.0

    # group trades by exec_date for fast daily application
    trades_by_day = {d: g for d, g in df.groupby("exec_date")}

    cash = float(cash0)
    pos = {}  # ticker -> shares
    equity = []

    for d in date_index:
        # apply trades for day d
        g = trades_by_day.get(pd.Timestamp(d).normalize())
        if g is not None:
            for _, r in g.iterrows():
                t = r["ticker"]
                side = r["side"]
                sh = int(r["shares"])
                px = r["fill_price"]
                fee = float(r["broker_fee"]) if pd.notna(r["broker_fee"]) else 0.0
                if not t or sh <= 0 or pd.isna(px) or px <= 0:
                    continue

                if side == "BUY":
                    cash -= (sh * float(px)) + fee
                    pos[t] = pos.get(t, 0) + sh
                elif side == "SELL":
                    sell_sh = min(sh, pos.get(t, 0))
                    if sell_sh <= 0:
                        continue
                    cash += (sell_sh * float(px)) - fee
                    pos[t] = pos.get(t, 0) - sell_sh
                    if pos[t] <= 0:
                        pos.pop(t, None)

        # mark-to-market
        mv = 0.0
        for t, sh in pos.items():
            arr = px_by_ticker.get(t)
            if arr is None:
                continue
            p = fast_price_lookup(arr, d)
            if pd.notna(p):
                mv += float(p) * float(sh)

        equity.append(cash + mv)

    return pd.Series(equity, index=date_index, name="strategy_equity")

def build_strategy_state_curves(exec_df: pd.DataFrame, px_by_ticker: dict, date_index: pd.DatetimeIndex) -> pd.DataFrame:
    """
    Builds daily end-of-day curves:
      - cash (uses audited cash_after when available)
      - securities_value (MTM using close_adj via px_by_ticker)
      - total_value = cash + securities_value

    Trades are applied intraday; then we overwrite cash with the last audited cash_after for that day (if present),
    so cash matches broker/ledger reconciliation when possible.
    """
    if exec_df is None or exec_df.empty or len(date_index) == 0:
        return pd.DataFrame(columns=["cash", "securities_value", "total_value"])

    df = exec_df.copy()
    df["exec_ts"] = _to_dt(df.get("exec_ts"))
    df["exec_date"] = _to_dt(df.get("exec_date")).dt.normalize()
    df["ticker"] = _norm_ticker(df.get("ticker", ""))
    df["side"] = _norm_side(df.get("side", ""))
    df["shares"] = pd.to_numeric(df.get("shares", 0), errors="coerce").fillna(0).astype(int)
    df["fill_price"] = pd.to_numeric(df.get("fill_price", np.nan), errors="coerce")
    df["broker_fee"] = pd.to_numeric(df.get("broker_fee", 0.0), errors="coerce").fillna(0.0)

    # audited cash columns (optional)
    if "cash_before" in df.columns:
        df["cash_before"] = pd.to_numeric(df["cash_before"], errors="coerce")
    if "cash_after" in df.columns:
        df["cash_after"] = pd.to_numeric(df["cash_after"], errors="coerce")

    # sort trades
    # sort trades: prefer ledger order; fallback if missing
    if "_ledger_seq" in df.columns:
        df = df.sort_values("_ledger_seq", kind="mergesort")
    else:
        df["_boid_num"] = pd.to_numeric(df.get("broker_order_id"), errors="coerce")
        if df["exec_ts"].notna().any():
            df = df.sort_values(["exec_ts", "_boid_num", "broker_order_id"], kind="mergesort")
        else:
            df = df.sort_values(["exec_date", "_boid_num", "broker_order_id"], kind="mergesort")
        df = df.drop(columns=["_boid_num"])


    # start cash
    cash0 = np.nan
    if "cash_before" in df.columns and df["cash_before"].notna().any():
        cash0 = float(df["cash_before"].dropna().iloc[0])
    elif "cash_after" in df.columns and df["cash_after"].notna().any():
        cash0 = float(df["cash_after"].dropna().iloc[0])
    if np.isnan(cash0):
        cash0 = 0.0

    trades_by_day = {d: g for d, g in df.groupby("exec_date")}

    cash = float(cash0)
    pos = {}  # ticker -> shares

    rows = []
    for d in date_index:
        d0 = pd.Timestamp(d).normalize()

        g = trades_by_day.get(d0)
        if g is not None and not g.empty:
            # apply trades to cash/pos
            for _, r in g.iterrows():
                t = r["ticker"]
                side = r["side"]
                sh = int(r["shares"])
                px = r["fill_price"]
                fee = float(r["broker_fee"]) if pd.notna(r["broker_fee"]) else 0.0

                if not t or sh <= 0 or pd.isna(px) or px <= 0:
                    continue

                if side == "BUY":
                    cash -= (sh * float(px)) + fee
                    pos[t] = pos.get(t, 0) + sh
                elif side == "SELL":
                    sell_sh = min(sh, pos.get(t, 0))
                    if sell_sh <= 0:
                        continue
                    cash += (sell_sh * float(px)) - fee
                    pos[t] = pos.get(t, 0) - sell_sh
                    if pos[t] <= 0:
                        pos.pop(t, None)

            # overwrite with audited cash_after if present (last trade of the day)
            # overwrite with audited cash_after if present (last trade of the day)
            if "cash_after" in g.columns and g["cash_after"].notna().any():
                if "_ledger_seq" in g.columns:
                    g = g.sort_values("_ledger_seq", kind="mergesort")
                audited_cash = float(g["cash_after"].dropna().iloc[-1])
                cash = audited_cash  # keep future days consistent too

        # mark-to-market securities value
        mv = 0.0
        for t, sh in pos.items():
            arr = px_by_ticker.get(t)
            if arr is None:
                continue
            p = fast_price_lookup(arr, d0)
            if pd.notna(p):
                mv += float(p) * float(sh)

        total = cash + mv
        rows.append({"date": d0, "cash": cash, "securities_value": mv, "total_value": total})

    out = pd.DataFrame(rows).set_index("date")
    return out


def perf_metrics_from_equity(equity: pd.Series) -> dict:
    """
    Returns YTD metrics:
      - return (simple)
      - max drawdown
      - sharpe (annualized)
      - sortino (annualized)
      - calmar (annualized_return / max_dd)
    """
    if equity is None or equity.empty:
        return {"ret": np.nan, "max_dd": np.nan, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}

    eq = pd.to_numeric(equity, errors="coerce").dropna()
    if len(eq) < 2:
        return {"ret": 0.0, "max_dd": 0.0, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}

    rets = eq.pct_change().dropna()
    if rets.empty:
        return {"ret": 0.0, "max_dd": 0.0, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}

    total_ret = float(eq.iloc[-1] / eq.iloc[0] - 1.0)

    roll_max = eq.cummax()
    dd = (eq / roll_max) - 1.0
    max_dd = float(dd.min())  # negative number (e.g. -0.12)

    mu = float(rets.mean())
    sd = float(rets.std(ddof=0))
    sharpe = (np.sqrt(TRADING_DAYS_PER_YEAR) * mu / sd) if sd > 0 else np.nan

    neg = rets[rets < 0]
    sd_neg = float(neg.std(ddof=0)) if len(neg) > 0 else 0.0
    sortino = (np.sqrt(TRADING_DAYS_PER_YEAR) * mu / sd_neg) if sd_neg > 0 else np.nan

    # annualized return over observed days
    n = len(rets)
    ann_ret = ((1.0 + total_ret) ** (TRADING_DAYS_PER_YEAR / max(1, n))) - 1.0
    calmar = (ann_ret / abs(max_dd)) if (pd.notna(max_dd) and max_dd < 0) else np.nan

    return {"ret": total_ret, "max_dd": max_dd, "sharpe": sharpe, "sortino": sortino, "calmar": calmar}

def save_perf_chart(path: str, dates: pd.DatetimeIndex, strat_eq: pd.Series, spy_eq: pd.Series, title: str):
    fig = plt.figure(figsize=(10, 3.2))
    ax = plt.gca()

    ok = True
    if dates is None or len(dates) < 2:
        ok = False
    if strat_eq is None or strat_eq.dropna().shape[0] < 2:
        ok = False
    if spy_eq is None or spy_eq.dropna().shape[0] < 2:
        ok = False

    if not ok:
        ax.text(0.5, 0.5, "NO DATA TO PLOT", ha="center", va="center")
        ax.set_title(title)
        ax.set_axis_off()
        fig.tight_layout()
        fig.savefig(path, dpi=150)
        plt.close(fig)
        return

    # align + normalize to 1.0 at start
    s1 = pd.to_numeric(strat_eq.reindex(dates), errors="coerce").dropna()
    s2 = pd.to_numeric(spy_eq.reindex(dates), errors="coerce").dropna()
    common = s1.index.intersection(s2.index)

    if len(common) < 2:
        ax.text(0.5, 0.5, "NO DATA TO PLOT", ha="center", va="center")
        ax.set_title(title)
        ax.set_axis_off()
        fig.tight_layout()
        fig.savefig(path, dpi=150)
        plt.close(fig)
        return

    s1 = s1.loc[common]
    s2 = s2.loc[common]

    s1n = s1 / float(s1.iloc[0])
    s2n = s2 / float(s2.iloc[0])

    ax.plot(common, s1n, marker="o", label="Strategy")
    ax.plot(common, s2n, marker="o", label="SPY")
    ax.legend()
    ax.set_title(title)
    ax.grid(alpha=0.3)

    fig.tight_layout()
    fig.savefig(path, dpi=150)
    plt.close(fig)


# ============================================================
# Load data
# ============================================================

exec_df = _safe_read_csv(EXECUTED_TRADES_FILE)
# Preserve the physical row order of executed_trades.csv (append-only ledger order)
exec_df["_ledger_seq"] = np.arange(len(exec_df), dtype=int)

recon_log = _safe_read_csv(RECON_LOG_FILE)
fills_df  = _safe_read_csv(MANUAL_FILLS_FILE)
live_cash, live_pos = read_live_portfolio_snapshot(LIVE_PORTFOLIO_FILE)

if exec_df.empty:
    raise FileNotFoundError(
        f"Missing or empty executed trades ledger: {EXECUTED_TRADES_FILE}\n"
        f"Run your reconciliation script at least once."
    )

# normalize columns/types
exec_df["exec_ts"] = _to_dt(exec_df.get("exec_ts"))
exec_df["exec_date"] = _to_dt(exec_df.get("exec_date"))
exec_df["signal_date"] = _to_dt(exec_df.get("signal_date"))

exec_df["ticker"] = _norm_ticker(exec_df.get("ticker", ""))
exec_df["side"] = _norm_side(exec_df.get("side", ""))
exec_df["broker_order_id"] = exec_df.get("broker_order_id", "").astype(str).str.strip()

for c in ("shares", "broker_fee", "gross_notional", "net_cash_impact", "order_price", "fill_price", "signal_price", "cash_before", "cash_after"):
    if c in exec_df.columns:
        exec_df[c] = pd.to_numeric(exec_df[c], errors="coerce")

# attach slope_rank so the table can sort by it
# attach slope_rank so the table can sort by it
exec_df = attach_slope_rank(exec_df)

# ============================================================
# STEP 3: Load rankings data (around line 715, after exec_df load)
# ============================================================
# ADD after the line: exec_df = attach_slope_rank(exec_df)

# NEW: Load master rankings
rankings_df = _safe_read_csv(MASTER_RANKINGS_FILE)

# NEW: Normalize rankings data
if not rankings_df.empty:
    rankings_df["signal_date"] = _to_dt(rankings_df.get("signal_date"))
    rankings_df["exec_date"] = _to_dt(rankings_df.get("exec_date"))
    rankings_df["ticker"] = _norm_ticker(rankings_df.get("ticker", ""))
    
    for c in ("slope_rank", "slope_adj", "atr20", "close_adj", "raw_weight", "capped_weight",
              "target_value", "target_weight", "target_shares", "current_shares",
              "current_value", "current_weight", "weight_change", "shares_change"):
        if c in rankings_df.columns:
            rankings_df[c] = pd.to_numeric(rankings_df[c], errors="coerce")

# Group rankings by signal_date for weekly lookup
rankings_by_signal_date = {}
if not rankings_df.empty and "signal_date" in rankings_df.columns:
    rankings_by_signal_date = {
        d: sub for d, sub in rankings_df.groupby(rankings_df["signal_date"].dt.normalize())
    }


# IMPORTANT: merges can reorder rows; restore append-only ledger order
if "_ledger_seq" in exec_df.columns:
    exec_df = exec_df.sort_values("_ledger_seq", kind="mergesort").reset_index(drop=True)


# ============================================================
# Build BOTH slippage columns (into SIG_SLIP_COL / ORD_SLIP_COL)
# ============================================================

side = _norm_side(exec_df["side"])
sh   = pd.to_numeric(exec_df.get("shares", np.nan), errors="coerce")

exec_df[SIG_SLIP_COL] = np.nan
if {"signal_price","fill_price","shares","side"}.issubset(exec_df.columns):
    s_px = pd.to_numeric(exec_df["signal_price"], errors="coerce")
    f_px = pd.to_numeric(exec_df["fill_price"], errors="coerce")
    exec_df[SIG_SLIP_COL] = np.where(
        side == "BUY",  (f_px - s_px) * sh,
        np.where(side == "SELL", (s_px - f_px) * sh, np.nan)
    )

exec_df[ORD_SLIP_COL] = np.nan
if {"order_price","fill_price","shares","side"}.issubset(exec_df.columns):
    o_px = pd.to_numeric(exec_df["order_price"], errors="coerce")
    f_px = pd.to_numeric(exec_df["fill_price"], errors="coerce")
    exec_df[ORD_SLIP_COL] = np.where(
        side == "BUY",  (f_px - o_px) * sh,
        np.where(side == "SELL", (o_px - f_px) * sh, np.nan)
    )

FIRST_EXEC_DATE = exec_df["exec_date"].min()
LAST_EXEC_DATE  = exec_df["exec_date"].max()

# ============================================================
# Load price DB and build fast lookup by ticker
# ============================================================

price_asof_date = pd.NaT
px_by_ticker = {}
try:
    if os.path.exists(PRICE_DB_FILE):
        price_df = pd.read_parquet(PRICE_DB_FILE)
        if PRICE_DATE_COL in price_df.columns and PRICE_TICKER_COL in price_df.columns and PRICE_PX_COL in price_df.columns:
            price_df[PRICE_DATE_COL] = pd.to_datetime(price_df[PRICE_DATE_COL], errors="coerce")
            price_df[PRICE_TICKER_COL] = price_df[PRICE_TICKER_COL].astype(str).str.upper().str.strip()
            price_df[PRICE_PX_COL] = pd.to_numeric(price_df[PRICE_PX_COL], errors="coerce")
            price_df = price_df.dropna(subset=[PRICE_DATE_COL, PRICE_TICKER_COL]).sort_values([PRICE_TICKER_COL, PRICE_DATE_COL])
            price_asof_date = price_df[PRICE_DATE_COL].max()

            for t, sub in price_df.groupby(PRICE_TICKER_COL, sort=False):
                sub = sub.sort_values(PRICE_DATE_COL)
                arr = np.zeros(len(sub), dtype=[("date","datetime64[ns]"), ("px","float64")])
                arr["date"] = sub[PRICE_DATE_COL].values.astype("datetime64[ns]")
                arr["px"] = sub[PRICE_PX_COL].astype(float).values
                px_by_ticker[t] = arr
        else:
            price_df = pd.DataFrame()
    else:
        price_df = pd.DataFrame()
except Exception:
    price_df = pd.DataFrame()
    
# ============================================================
# ASOF (needed early) + Load SPY close series (needed for state_curves)
# ============================================================

# Use the later of (price DB last date) and (last exec date) so state_curves includes all exec days.
_asof_price = price_asof_date if pd.notna(price_asof_date) else pd.NaT
_asof_exec  = LAST_EXEC_DATE.normalize() if pd.notna(LAST_EXEC_DATE) else pd.NaT

if pd.notna(_asof_price) and pd.notna(_asof_exec):
    asof = max(_asof_price, _asof_exec)
elif pd.notna(_asof_price):
    asof = _asof_price
elif pd.notna(_asof_exec):
    asof = _asof_exec
else:
    asof = pd.Timestamp.today().normalize()


spy_close = load_spy_close_series()
spy_last_date = spy_close.index.max() if not spy_close.empty else pd.NaT

# ============================================================
# Trading calendar for valuation-date logic
# ============================================================

if "price_df" in globals() and isinstance(price_df, pd.DataFrame) and (not price_df.empty) and (PRICE_DATE_COL in price_df.columns):
    _cal = pd.to_datetime(price_df[PRICE_DATE_COL], errors="coerce").dropna().dt.normalize().unique()
    TRADING_CAL = pd.DatetimeIndex(sorted(_cal))
elif spy_close is not None and not spy_close.empty:
    TRADING_CAL = pd.DatetimeIndex(sorted(pd.to_datetime(spy_close.index).normalize().unique()))
else:
    TRADING_CAL = pd.DatetimeIndex(sorted(exec_dates_norm.unique()))


# ============================================================
# NEW: Daily state curves for per-week portfolio & cash blocks
# ============================================================

state_curves = pd.DataFrame()

# Make sure ALL exec dates exist in the date index (even if SPY is missing that date)
exec_dates_norm = exec_df["exec_date"].dropna().dt.normalize()
base_dates = pd.DatetimeIndex([])

if not spy_close.empty:
    base_dates = spy_close.index
else:
    # fallback: use exec dates only
    base_dates = exec_dates_norm.sort_values().unique()

# union to guarantee coverage
all_dates = pd.DatetimeIndex(sorted(set(pd.to_datetime(base_dates).normalize()) | set(exec_dates_norm)))

# optional clamp to a reasonable range
# optional clamp to a reasonable range
first_needed = pd.Timestamp(FIRST_EXEC_DATE).normalize()
last_exec_needed = exec_dates_norm.max() if len(exec_dates_norm) else pd.Timestamp(asof).normalize()
last_needed = max(pd.Timestamp(asof).normalize(), pd.Timestamp(last_exec_needed).normalize())

all_dates = all_dates[(all_dates >= first_needed) & (all_dates <= last_needed)]


if len(all_dates) >= 1:
    state_curves = build_strategy_state_curves(exec_df, px_by_ticker, all_dates)



# ============================================================
# Reconciliation quality checks
# ============================================================

dup_counts = exec_df["broker_order_id"].value_counts(dropna=False)
dups = dup_counts[dup_counts > 1]
dup_df = (
    exec_df[exec_df["broker_order_id"].isin(dups.index)]
    .sort_values(["broker_order_id", "exec_ts"], kind="mergesort")
    if not dups.empty else pd.DataFrame()
)

pending_df = pd.DataFrame()
if not fills_df.empty and "broker_order_id" in fills_df.columns:
    fills_df["broker_order_id"] = fills_df["broker_order_id"].astype(str).str.strip()
    executed_ids = set(exec_df["broker_order_id"].astype(str))
    pending_df = fills_df[~fills_df["broker_order_id"].isin(executed_ids)].copy()

ledger_pos = reconstruct_positions_from_ledger(exec_df)

all_tickers = sorted(set(ledger_pos.keys()) | set(live_pos.keys()))
drift_rows = []
for t in all_tickers:
    drift_rows.append({
        "ticker": t,
        "ledger_shares": ledger_pos.get(t, 0),
        "live_shares": live_pos.get(t, 0),
        "diff_shares": live_pos.get(t, 0) - ledger_pos.get(t, 0),
    })
drift_df = pd.DataFrame(drift_rows)

ledger_cash = float(exec_df["cash_after"].dropna().iloc[-1]) if "cash_after" in exec_df.columns and exec_df["cash_after"].notna().any() else np.nan
cash_drift = (live_cash - ledger_cash) if (not np.isnan(live_cash) and not np.isnan(ledger_cash)) else np.nan

# ============================================================
# Build CURRENT portfolio table with avg cost + current price + returns
# ============================================================

basis_map = compute_cost_basis_from_ledger(exec_df)

port_rows = []
# asof already defined above (after price DB load)

for t, sh_ in live_pos.items():
    row = {
        "ticker": t,
        "shares": int(sh_),
        "avg_cost": np.nan,
        "first_acquired": pd.NaT,
        "current_price": np.nan,
        "market_value": np.nan,
        "weight_pct": np.nan,
        "position_return_pct": np.nan,
    }

    if t in basis_map:
        row["avg_cost"] = basis_map[t]["avg_cost"]
        row["first_acquired"] = basis_map[t]["first_acquired"]

    if t in px_by_ticker and pd.notna(asof):
        row["current_price"] = fast_price_lookup(px_by_ticker[t], asof)

    if pd.notna(row["current_price"]):
        row["market_value"] = row["current_price"] * row["shares"]

    if pd.notna(row["avg_cost"]) and row["avg_cost"] > 0 and pd.notna(row["current_price"]):
        row["position_return_pct"] = (row["current_price"] / row["avg_cost"] - 1.0) * 100.0

    port_rows.append(row)

port_df = pd.DataFrame(port_rows)
if not port_df.empty:
    total_mv = float(port_df["market_value"].sum(skipna=True))
    port_df["weight_pct"] = np.where(total_mv > 0, (port_df["market_value"] / total_mv) * 100.0, np.nan)
    port_df = port_df.sort_values("weight_pct", ascending=False, na_position="last").reset_index(drop=True)

if drift_df is not None and not drift_df.empty:
    drift_df = drift_df.merge(port_df[["ticker", "weight_pct"]], on="ticker", how="left")
    drift_df = drift_df[drift_df["diff_shares"] != 0].sort_values("ticker") if not drift_df.empty else drift_df

# ============================================================
# Weekly aggregation (by exec_date)
# ============================================================

weekly = exec_df.copy()
weekly = weekly[weekly["exec_date"].notna()].copy()
weekly["exec_date_only"] = weekly["exec_date"].dt.normalize()
weekly_grp = weekly.groupby("exec_date_only", dropna=True)

weekly_summary = weekly_grp.agg(
    trades=("broker_order_id", "count"),
    buys=("side", lambda s: int((_norm_side(s) == "BUY").sum())),
    sells=("side", lambda s: int((_norm_side(s) == "SELL").sum())),
    gross_notional=("gross_notional", "sum"),
    fees=("broker_fee", "sum"),
    missing_signal_price=("signal_price", lambda s: int(pd.to_numeric(s, errors="coerce").isna().sum())) if "signal_price" in weekly.columns else ("broker_order_id", lambda s: 0),
    missing_order_price=("order_price", lambda s: int(pd.to_numeric(s, errors="coerce").isna().sum())) if "order_price" in weekly.columns else ("broker_order_id", lambda s: 0),
).reset_index().rename(columns={"exec_date_only": "exec_date"})

def _slip_apply(g: pd.DataFrame) -> pd.Series:
    sig = slippage_breakdown(g, slip_col=SIG_SLIP_COL, notional_col="gross_notional")
    ord_ = slippage_breakdown(g, slip_col=ORD_SLIP_COL, notional_col="gross_notional")
    return pd.Series({
        "sig_slip_net_dollars": sig["net_dollars"],
        "sig_slip_gross_cost_dollars": sig["gross_cost_dollars"],
        "sig_slip_gross_improve_dollars": sig["gross_improve_dollars"],
        "sig_slip_net_bps": sig["net_bps"],
        "ord_slip_net_dollars": ord_["net_dollars"],
        "ord_slip_gross_cost_dollars": ord_["gross_cost_dollars"],
        "ord_slip_gross_improve_dollars": ord_["gross_improve_dollars"],
        "ord_slip_net_bps": ord_["net_bps"],
    })

slip_by_day = weekly_grp.apply(_slip_apply).reset_index().rename(columns={"exec_date_only": "exec_date"})
weekly_summary = weekly_summary.merge(slip_by_day, on="exec_date", how="left")

weekly_summary = weekly_summary[
    (weekly_summary["exec_date"].dt.year >= MIN_YEAR_FOR_REPORT) &
    (weekly_summary["exec_date"].dt.year <= MAX_YEAR_FOR_REPORT)
].sort_values("exec_date")


# ============================================================
# NEW: previous execution-day mapping (for weekly portfolio return)
# ============================================================

_exec_days = (
    pd.to_datetime(weekly_summary["exec_date"], errors="coerce")
      .dropna()
      .dt.normalize()
      .sort_values()
      .tolist()
)

prev_exec_map = {d: (_exec_days[i-1] if i > 0 else pd.NaT) for i, d in enumerate(_exec_days)}


def ytd_slice(dt: pd.Timestamp) -> pd.DataFrame:
    start = max(pd.Timestamp(dt.year, 1, 1), FIRST_EXEC_DATE.normalize())
    return weekly[(weekly["exec_date_only"] >= start) & (weekly["exec_date_only"] <= dt)]

# ============================================================
# Charts (existing)
# ============================================================

chart_dir = OUTPUT_DIR
fee_chart_path = os.path.join(chart_dir, "cum_fees.png")
slip_chart_path = os.path.join(chart_dir, "cum_slippage_breakout.png")
count_chart_path = os.path.join(chart_dir, "trade_counts.png")

ws = weekly_summary.copy()
ws = ws[ws["exec_date"].notna()].copy()
ws = ws.sort_values("exec_date")

plot_cols = [
    "fees","trades",
    "sig_slip_net_dollars","sig_slip_gross_cost_dollars","sig_slip_gross_improve_dollars",
    "ord_slip_net_dollars","ord_slip_gross_cost_dollars","ord_slip_gross_improve_dollars",
]
for c in plot_cols:
    if c not in ws.columns:
        ws[c] = 0.0
    ws[c] = pd.to_numeric(ws[c], errors="coerce").fillna(0.0)

if ws.empty:
    for path, title in [
        (fee_chart_path, "Cumulative Broker Fees (no data)"),
        (slip_chart_path, "Cumulative Slippage (no data)"),
        (count_chart_path, "Trades per Execution Day (no data)"),
    ]:
        fig = plt.figure(figsize=(10, 3.2))
        ax = plt.gca()
        ax.text(0.5, 0.5, "NO DATA TO PLOT", ha="center", va="center")
        ax.set_title(title)
        ax.set_axis_off()
        fig.tight_layout()
        fig.savefig(path, dpi=150)
        plt.close(fig)
else:
    # Fees
    fig = plt.figure(figsize=(10, 3.2))
    ax = plt.gca()
    ax.plot(ws["exec_date"], ws["fees"].cumsum(), marker="o")
    ax.set_title("Cumulative Broker Fees (Ledger)")
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(fee_chart_path, dpi=150)
    plt.close(fig)

    # Slippage
    fig = plt.figure(figsize=(10, 3.2))
    ax = plt.gca()
    ax.plot(ws["exec_date"], ws["sig_slip_net_dollars"].cumsum(), marker="o", label="SigFill Net ($)")
    ax.plot(ws["exec_date"], ws["ord_slip_net_dollars"].cumsum(), marker="o", label="OrdFill Net ($)")
    ax.plot(ws["exec_date"], ws["sig_slip_gross_cost_dollars"].cumsum(), marker="o", label="SigFill Gross Cost ($)")
    ax.plot(ws["exec_date"], ws["sig_slip_gross_improve_dollars"].cumsum(), marker="o", label="SigFill Gross Improve ($)")
    ax.plot(ws["exec_date"], ws["ord_slip_gross_cost_dollars"].cumsum(), marker="o", label="OrdFill Gross Cost ($)")
    ax.plot(ws["exec_date"], ws["ord_slip_gross_improve_dollars"].cumsum(), marker="o", label="OrdFill Gross Improve ($)")
    ax.legend()
    ax.set_title("Cumulative Slippage — SigFill vs OrdFill (Net + Gross Cost/Improve)")
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(slip_chart_path, dpi=150)
    plt.close(fig)

    # Trade counts
    fig = plt.figure(figsize=(10, 3.2))
    ax = plt.gca()
    ax.plot(ws["exec_date"], ws["trades"], marker="o")
    ax.set_title("Trades per Execution Day (Ledger)")
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(count_chart_path, dpi=150)
    plt.close(fig)

# ============================================================
# PDF Styles
# ============================================================

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="Small",  fontSize=8, leading=9))
styles.add(ParagraphStyle(name="Tiny",   fontSize=7, leading=8))
styles.add(ParagraphStyle(name="Header", fontSize=12, leading=14, spaceAfter=6, spaceBefore=6))
styles.add(ParagraphStyle(name="TitleBig", fontSize=16, leading=20, spaceAfter=10))

# ============================================================
# NEW: Build YTD Performance Table (Strategy vs SPY)
# ============================================================
# ============================================================
# NEW: Build YTD + WEEKLY Performance (Strategy vs SPY) + Charts
# ============================================================

WEEKLY_LOOKBACK_TRADING_DAYS = 5

# choose as-of for perf table: must exist in SPY data if possible
asof_perf = asof
if pd.notna(spy_last_date):
    asof_perf = min(pd.Timestamp(asof).normalize(), pd.Timestamp(spy_last_date).normalize())

# pick YTD start: max(Jan1, first exec date, first spy date)
if pd.notna(asof_perf):
    jan1 = pd.Timestamp(asof_perf.year, 1, 1)
else:
    jan1 = pd.Timestamp.today().normalize().replace(month=1, day=1)

first_exec_norm = pd.Timestamp(FIRST_EXEC_DATE).normalize() if pd.notna(FIRST_EXEC_DATE) else jan1
first_spy_norm  = pd.Timestamp(spy_close.index.min()).normalize() if not spy_close.empty else jan1
ytd_start = max(jan1, first_exec_norm, first_spy_norm)

# Build a FULL curve (so YTD/Weekly slices start from correct portfolio state)
curve_dates = pd.DatetimeIndex([])
strat_eq_full = pd.Series(dtype=float)
spy_eq_full = pd.Series(dtype=float)

if not spy_close.empty and pd.notna(asof_perf):
    curve_start = max(first_exec_norm, first_spy_norm)  # ensures we have SPY dates
    curve_dates = spy_close.loc[(spy_close.index >= curve_start) & (spy_close.index <= asof_perf)].index

    if len(curve_dates) >= 2:
        strat_eq_full = build_strategy_equity_curve(exec_df, px_by_ticker, curve_dates)
        spy_eq_full = spy_close.loc[curve_dates].astype(float)

# ---- YTD metrics ----
strategy_metrics = {"ret": np.nan, "max_dd": np.nan, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}
spy_metrics      = {"ret": np.nan, "max_dd": np.nan, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}

ytd_dates = pd.DatetimeIndex([])
if len(curve_dates) >= 2:
    ytd_dates = curve_dates[curve_dates >= ytd_start]
    if len(ytd_dates) >= 2:
        strategy_metrics = perf_metrics_from_equity(strat_eq_full.loc[ytd_dates])
        spy_metrics      = perf_metrics_from_equity(spy_eq_full.loc[ytd_dates])

# ---- WEEKLY metrics (last 5 SPY trading sessions ending at asof_perf) ----
weekly_metrics_strat = {"ret": np.nan, "max_dd": np.nan, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}
weekly_metrics_spy   = {"ret": np.nan, "max_dd": np.nan, "sharpe": np.nan, "sortino": np.nan, "calmar": np.nan}

weekly_dates = pd.DatetimeIndex([])
weekly_start = pd.NaT
if len(curve_dates) >= 2:
    weekly_dates = curve_dates[-WEEKLY_LOOKBACK_TRADING_DAYS:] if len(curve_dates) >= WEEKLY_LOOKBACK_TRADING_DAYS else curve_dates
    if len(weekly_dates) >= 2:
        weekly_start = pd.Timestamp(weekly_dates.min()).normalize()
        weekly_metrics_strat = perf_metrics_from_equity(strat_eq_full.loc[weekly_dates])
        weekly_metrics_spy   = perf_metrics_from_equity(spy_eq_full.loc[weekly_dates])

# ---- Performance charts (saved alongside your other charts) ----
ytd_perf_chart_path    = os.path.join(OUTPUT_DIR, "perf_ytd_strategy_vs_spy.png")
weekly_perf_chart_path = os.path.join(OUTPUT_DIR, "perf_weekly_strategy_vs_spy.png")

# always write chart files (even if "NO DATA")
save_perf_chart(
    ytd_perf_chart_path,
    ytd_dates,
    strat_eq_full if len(curve_dates) else pd.Series(dtype=float),
    spy_eq_full if len(curve_dates) else pd.Series(dtype=float),
    f"YTD Strategy vs SPY (Normalized) — {ytd_start.date()} → {pd.Timestamp(asof_perf).date() if pd.notna(asof_perf) else 'N/A'}"
)

wk_title_start = pd.Timestamp(weekly_start).date() if pd.notna(weekly_start) else "N/A"
save_perf_chart(
    weekly_perf_chart_path,
    weekly_dates,
    strat_eq_full if len(curve_dates) else pd.Series(dtype=float),
    spy_eq_full if len(curve_dates) else pd.Series(dtype=float),
    f"Weekly Strategy vs SPY (Normalized, last {WEEKLY_LOOKBACK_TRADING_DAYS} trading sessions) — {wk_title_start} → {pd.Timestamp(asof_perf).date() if pd.notna(asof_perf) else 'N/A'}"
)



# ============================================================
# Build PDF
# ============================================================

pdf_name = f"reconciliation_report_{datetime.now():%Y%m%d-%H%M%S}.pdf"
pdf_path = os.path.join(OUTPUT_DIR, pdf_name)

doc = SimpleDocTemplate(
    pdf_path,
    pagesize=landscape(letter),
    rightMargin=36, leftMargin=36,
    topMargin=36,  bottomMargin=36,
)

story = []

# ---------- Front Page ----------
story.append(Paragraph("Reconciliation Report (Executed Ledger + Diagnostics)", styles["TitleBig"]))
story.append(Paragraph(f"Generated: {datetime.now().isoformat(timespec='seconds')}", styles["Small"]))
story.append(Spacer(1, 0.10 * inch))

front_rows = [
    ["Item", "Value"],
    ["Ledger rows (executed_trades.csv)", f"{len(exec_df):,}"],
    ["First exec date", _fmt(FIRST_EXEC_DATE)],
    ["Last exec date", _fmt(LAST_EXEC_DATE)],
    ["Unique broker_order_id", f"{exec_df['broker_order_id'].nunique(dropna=True):,}"],
    ["Duplicate broker_order_id (count)", f"{int((exec_df['broker_order_id'].duplicated()).sum()):,}"],
    ["Pending fills (manual not in ledger)", f"{len(pending_df):,}"],
    ["Live cash (snapshot)", "N/A" if np.isnan(live_cash) else f"{live_cash:,.2f}"],
    ["Ledger cash (last cash_after)", "N/A" if np.isnan(ledger_cash) else f"{ledger_cash:,.2f}"],
    ["Cash drift (live - ledger)", "N/A" if np.isnan(cash_drift) else f"{cash_drift:,.2f}"],
    ["Position drift tickers (count)", f"{0 if drift_df is None or drift_df.empty else len(drift_df):,}"],
    ["Price DB as-of date", "N/A" if pd.isna(asof) else str(pd.Timestamp(asof).date())],
    ["Slope-rank source (master_trades.csv)", "FOUND" if os.path.exists(MASTER_TRADES_FILE) else "MISSING"],
    ["Rankings source (master_rankings.csv)", "FOUND" if os.path.exists(MASTER_RANKINGS_FILE) else "MISSING"],
    ["SPY regime file", "FOUND" if (os.path.exists(SPY_REGIME_FILE_PRIMARY) or os.path.exists(SPY_REGIME_FILE_FALLBACK)) else "MISSING"],
]
tbl = Table(front_rows, hAlign="LEFT")
tbl.setStyle(TableStyle([
    ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
    ("GRID",(0,0),(-1,-1),0.25,colors.grey),
    ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
    ("FONTSIZE",(0,0),(-1,-1),9),
    ("ALIGN",(1,1),(-1,-1),"RIGHT"),
]))
story.append(tbl)
story.append(Spacer(1, 0.12 * inch))

# ---------- NEW: Strategy vs SPY Performance Table (like screenshot) ----------
# use YTD slippage based on SigFill (your report’s main slippage set)
# ---------- NEW: Strategy vs SPY Performance Table (like screenshot) ----------
# use YTD slippage based on SigFill (your report’s main slippage set)

ytd_weekly_slice = weekly
if pd.notna(ytd_start) and pd.notna(asof_perf):
    ytd_weekly_slice = weekly[
        (weekly["exec_date_only"] >= pd.Timestamp(ytd_start).normalize()) &
        (weekly["exec_date_only"] <= pd.Timestamp(asof_perf).normalize())
    ]

ytd_sig = slippage_breakdown(
    ytd_weekly_slice,
    slip_col=SIG_SLIP_COL,
    notional_col="gross_notional"
)


perf_table = [
    ["Metric", "Strategy", "SPY"],
    ["YTD Return", _fmt_pct(strategy_metrics["ret"]), _fmt_pct(spy_metrics["ret"])],
    ["YTD Max Drawdown", _fmt_pct(strategy_metrics["max_dd"]), _fmt_pct(spy_metrics["max_dd"])],
    ["YTD Sharpe", _fmt_num(strategy_metrics["sharpe"]), _fmt_num(spy_metrics["sharpe"])],
    ["YTD Sortino", _fmt_num(strategy_metrics["sortino"]), _fmt_num(spy_metrics["sortino"])],
    ["YTD Calmar", _fmt_num(strategy_metrics["calmar"]), _fmt_num(spy_metrics["calmar"])],
    ["YTD Slippage (net $)", _fmt(ytd_sig["net_dollars"], "ytd_sig_slip_net_dollars"), "N/A"],
    ["YTD Slippage (gross cost $)", _fmt(ytd_sig["gross_cost_dollars"], "ytd_sig_slip_gross_cost_dollars"), "N/A"],
    ["YTD Slippage (gross improve $)", _fmt(ytd_sig["gross_improve_dollars"], "ytd_sig_slip_gross_improve_dollars"), "N/A"],
    ["YTD Slippage (net bps)", _fmt(ytd_sig["net_bps"], "ytd_sig_slip_net_bps"), "N/A"],
    # keep these duplicate-style rows to match your screenshot feel
    ["YTD Slippage ($, cost)", _fmt(ytd_sig["net_dollars"], "ytd_sig_slip_net_dollars"), "N/A"],
    ["YTD Slippage (bps)", _fmt(ytd_sig["net_bps"], "ytd_sig_slip_net_bps"), "N/A"],
]

story.append(Paragraph(f"YTD Performance (from {ytd_start.date()} to {pd.Timestamp(asof_perf).date() if pd.notna(asof_perf) else 'N/A'})", styles["Header"]))
pt = Table(perf_table, hAlign="LEFT")
pt.setStyle(TableStyle([
    ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
    ("GRID",(0,0),(-1,-1),0.25,colors.grey),
    ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
    ("FONTSIZE",(0,0),(-1,-1),9),
    ("ALIGN",(1,1),(-1,-1),"RIGHT"),
]))
story.append(pt)
story.append(Spacer(1, 0.15 * inch))

# ---------- NEW: Weekly Performance Table (Strategy vs SPY) ----------
weekly_sig = slippage_breakdown(
    weekly[(weekly["exec_date_only"] >= weekly_start) & (weekly["exec_date_only"] <= asof_perf)]
    if (pd.notna(weekly_start) and pd.notna(asof_perf)) else weekly,
    slip_col=SIG_SLIP_COL,
    notional_col="gross_notional"
)

weekly_perf_table = [
    ["Metric", "Strategy", "SPY"],
    ["Weekly Return", _fmt_pct(weekly_metrics_strat["ret"]), _fmt_pct(weekly_metrics_spy["ret"])],
    ["Weekly Max Drawdown", _fmt_pct(weekly_metrics_strat["max_dd"]), _fmt_pct(weekly_metrics_spy["max_dd"])],
    ["Weekly Sharpe", _fmt_num(weekly_metrics_strat["sharpe"]), _fmt_num(weekly_metrics_spy["sharpe"])],
    ["Weekly Sortino", _fmt_num(weekly_metrics_strat["sortino"]), _fmt_num(weekly_metrics_spy["sortino"])],
    ["Weekly Calmar", _fmt_num(weekly_metrics_strat["calmar"]), _fmt_num(weekly_metrics_spy["calmar"])],
    ["Weekly Slippage (net $)", _fmt(weekly_sig["net_dollars"], "sig_slip_net_dollars"), "N/A"],
    ["Weekly Slippage (gross cost $)", _fmt(weekly_sig["gross_cost_dollars"], "sig_slip_gross_cost_dollars"), "N/A"],
    ["Weekly Slippage (gross improve $)", _fmt(weekly_sig["gross_improve_dollars"], "sig_slip_gross_improve_dollars"), "N/A"],
    ["Weekly Slippage (net bps)", _fmt(weekly_sig["net_bps"], "sig_slip_net_bps"), "N/A"],
]

wk_end = pd.Timestamp(asof_perf).date() if pd.notna(asof_perf) else "N/A"
wk_start = pd.Timestamp(weekly_start).date() if pd.notna(weekly_start) else "N/A"
story.append(Paragraph(f"Weekly Performance (from {wk_start} to {wk_end})", styles["Header"]))

wpt = Table(weekly_perf_table, hAlign="LEFT")
wpt.setStyle(TableStyle([
    ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
    ("GRID",(0,0),(-1,-1),0.25,colors.grey),
    ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
    ("FONTSIZE",(0,0),(-1,-1),9),
    ("ALIGN",(1,1),(-1,-1),"RIGHT"),
]))
story.append(wpt)
story.append(Spacer(1, 0.15 * inch))


# keep existing charts page (unchanged)
story.append(Paragraph("Overview Charts", styles["Header"]))

# NEW: performance charts (YTD + Weekly) above trades
story.append(Image(ytd_perf_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(Spacer(1, 0.10 * inch))
story.append(Image(weekly_perf_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(Spacer(1, 0.10 * inch))

# existing charts (unchanged)
story.append(Image(fee_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(Spacer(1, 0.10 * inch))
story.append(Image(slip_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(Spacer(1, 0.10 * inch))
story.append(Image(count_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(PageBreak())



story.append(Spacer(1, 0.10 * inch))
story.append(Image(slip_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(Spacer(1, 0.10 * inch))
story.append(Image(count_chart_path, width=9.0 * inch, height=2.6 * inch))
story.append(PageBreak())

# ---------- Global Diagnostics ----------
story.append(Paragraph("Global Diagnostics", styles["Header"]))

make_table(
    story,
    "Duplicate broker_order_id rows (should be NONE)",
    dup_df,
    cols=["exec_ts","exec_date","ticker","side","shares","signal_price","order_price","fill_price","broker_fee", SIG_SLIP_COL, ORD_SLIP_COL, "broker_order_id"],
    font_size=7
)

make_table(
    story,
    "Pending broker fills (in broker_fills_manual.csv but NOT in executed_trades.csv)",
    pending_df,
    cols=["plan_id","signal_date","exec_date","ticker","side","shares_filled","order_type","order_price","fill_price","broker_fee","broker_order_id"],
    font_size=7
)

make_table(
    story,
    "Portfolio Drift: live_portfolio.csv vs positions reconstructed from executed_trades.csv",
    drift_df,
    cols=["ticker","ledger_shares","live_shares","diff_shares","weight_pct"],
    font_size=8
)

story.append(PageBreak())

# ---------- Weekly Loop ----------
for _, wk in weekly_summary.iterrows():
    exec_dt = pd.Timestamp(wk["exec_date"]).normalize()
    
    if exec_dt.year < MIN_YEAR_FOR_REPORT or exec_dt.year > MAX_YEAR_FOR_REPORT:
        continue

    day = weekly[weekly["exec_date_only"] == exec_dt].copy()
    # Keep audited columns, but rename so tables don't imply they're sequential after sorting
    day = day.rename(columns={
        "cash_before": "cash_before_audited",
        "cash_after": "cash_after_audited",
    })

    # NEW: Get rankings for this week
    # signal_date is typically the day before exec_date
    signal_dt = None
    if not day.empty and "signal_date" in day.columns and day["signal_date"].notna().any():
        signal_dt = pd.Timestamp(day["signal_date"].dropna().iloc[0]).normalize()
    
    day_rankings = pd.DataFrame()
    if signal_dt is not None and signal_dt in rankings_by_signal_date:
        day_rankings = rankings_by_signal_date[signal_dt].copy()

    ytd = ytd_slice(exec_dt)

    day_sig = slippage_breakdown(day, slip_col=SIG_SLIP_COL, notional_col="gross_notional")
    ytd_sig_day = slippage_breakdown(ytd, slip_col=SIG_SLIP_COL, notional_col="gross_notional")

    day_ord = slippage_breakdown(day, slip_col=ORD_SLIP_COL, notional_col="gross_notional")
    ytd_ord = slippage_breakdown(ytd, slip_col=ORD_SLIP_COL, notional_col="gross_notional")

    ytd_trades = len(ytd)
    day_trades = int(wk["trades"]) if pd.notna(wk.get("trades")) else len(day)

    day_sig_per_trade = (day_sig["net_dollars"] / day_trades) if day_trades > 0 and pd.notna(day_sig["net_dollars"]) else np.nan
    ytd_sig_per_trade = (ytd_sig_day["net_dollars"] / ytd_trades) if ytd_trades > 0 and pd.notna(ytd_sig_day["net_dollars"]) else np.nan

    day_ord_per_trade = (day_ord["net_dollars"] / day_trades) if day_trades > 0 and pd.notna(day_ord["net_dollars"]) else np.nan
    ytd_ord_per_trade = (ytd_ord["net_dollars"] / ytd_trades) if ytd_trades > 0 and pd.notna(ytd_ord["net_dollars"]) else np.nan

    ytd_fees = float(ytd["broker_fee"].sum(skipna=True)) if "broker_fee" in ytd.columns else np.nan

    # -----------------------------
    # NEW: AUDITED CASH (EXECUTION DAY)
    # -----------------------------
    day_sorted_for_cash = day.copy()

    # Use ledger order first (best). Fallback to timestamp+numeric id if needed.
    if "_ledger_seq" in day_sorted_for_cash.columns:
        day_sorted_for_cash = day_sorted_for_cash.sort_values("_ledger_seq", kind="mergesort")
    else:
        day_sorted_for_cash["_boid_num"] = pd.to_numeric(day_sorted_for_cash.get("broker_order_id"), errors="coerce")
        if "exec_ts" in day_sorted_for_cash.columns and day_sorted_for_cash["exec_ts"].notna().any():
            day_sorted_for_cash = day_sorted_for_cash.sort_values(["exec_ts", "_boid_num", "broker_order_id"], kind="mergesort")
        else:
            day_sorted_for_cash = day_sorted_for_cash.sort_values(["exec_date", "_boid_num", "broker_order_id"], kind="mergesort")
        day_sorted_for_cash = day_sorted_for_cash.drop(columns=["_boid_num"])


    cash_before_first = np.nan
    if (not day_sorted_for_cash.empty) and ("cash_before_audited" in day_sorted_for_cash.columns) and day_sorted_for_cash["cash_before_audited"].notna().any():
        cash_before_first = float(pd.to_numeric(day_sorted_for_cash["cash_before_audited"], errors="coerce").dropna().iloc[0])

    cash_after_last = np.nan
    if (not day_sorted_for_cash.empty) and ("cash_after_audited" in day_sorted_for_cash.columns) and day_sorted_for_cash["cash_after_audited"].notna().any():
        cash_after_last = float(pd.to_numeric(day_sorted_for_cash["cash_after_audited"], errors="coerce").dropna().iloc[-1])

    cash_change = (cash_after_last - cash_before_first) if (pd.notna(cash_after_last) and pd.notna(cash_before_first)) else np.nan

    # -----------------------------
    # NEW: PORTFOLIO (AFTER TRADES, END OF DAY)
    # -----------------------------
    # -----------------------------
    # PORTFOLIO VALUATION LOGIC (TWO SNAPSHOTS)
    #   (1) Pre-trade valuation = prior trading day close
    #   (2) Post-trade valuation = execution day close (or best available if prices not present)
    # -----------------------------

    # ---- (1) PRE-TRADE snapshot date = prior trading day in state_curves ----
    prev_mkt_dt = pd.NaT
    if state_curves is not None and (not state_curves.empty):
        # previous trading day available in the curve (handles weekends/holidays)
        prev_candidates = state_curves.index[state_curves.index < exec_dt]
        if len(prev_candidates) > 0:
            prev_mkt_dt = pd.Timestamp(prev_candidates.max()).normalize()

    # defaults
    pre_portfolio_value = np.nan
    pre_securities_value = np.nan
    pre_cash_value = np.nan

    if pd.notna(prev_mkt_dt) and (state_curves is not None) and (not state_curves.empty) and (prev_mkt_dt in state_curves.index):
        pre_cash_value = float(state_curves.loc[prev_mkt_dt, "cash"])
        pre_securities_value = float(state_curves.loc[prev_mkt_dt, "securities_value"])
        if pd.notna(pre_cash_value) and pd.notna(pre_securities_value):
            pre_portfolio_value = pre_cash_value + pre_securities_value
        else:
            pre_portfolio_value = float(state_curves.loc[prev_mkt_dt, "total_value"])

    # ---- (2) POST-TRADE (end-of-day) snapshot = exec_dt close ----
    post_portfolio_value = np.nan
    post_securities_value = np.nan

    # prefer audited cash from the ledger day block
    post_cash_value = cash_after_last

    if (state_curves is not None) and (not state_curves.empty) and (exec_dt in state_curves.index):
        post_securities_value = float(state_curves.loc[exec_dt, "securities_value"])

        # if audited cash is missing, fall back to curve cash
        if pd.isna(post_cash_value):
            post_cash_value = float(state_curves.loc[exec_dt, "cash"])

        # total should match the cash we actually display (audited preferred)
        if pd.notna(post_cash_value) and pd.notna(post_securities_value):
            post_portfolio_value = post_cash_value + post_securities_value
        else:
            post_portfolio_value = float(state_curves.loc[exec_dt, "total_value"])

    # -----------------------------
    # WEEKLY PORTFOLIO RETURN (prev execution day close -> this execution day close)
    # -----------------------------
    prev_dt = prev_exec_map.get(exec_dt, pd.NaT)
    weekly_portfolio_return = np.nan

    if pd.notna(prev_dt) and (state_curves is not None) and (not state_curves.empty) and (prev_dt in state_curves.index):
        prev_total = float(state_curves.loc[prev_dt, "total_value"])
        cur_total = post_portfolio_value

        if pd.notna(prev_total) and prev_total > 0 and pd.notna(cur_total):
            weekly_portfolio_return = (cur_total / prev_total - 1.0)


    # -----------------------------
    # Render tables + summary
    # -----------------------------
    story.append(Paragraph(f"Execution Day: {exec_dt.date()} — Reconciliation Summary", styles["Header"]))
    story.append(Spacer(1, 0.08 * inch))

    audited_cash_rows = [
        ["Item", "Amount"],
        ["Cash BEFORE first trade", _fmt(cash_before_first, "cash")],
        ["Cash AFTER last trade", _fmt(cash_after_last, "cash")],
        ["Cash Δ (After - Before)", _fmt(cash_change, "cash")],
    ]
    t_cash = Table(audited_cash_rows, hAlign="LEFT")
    t_cash.setStyle(TableStyle([
        ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
        ("GRID",(0,0),(-1,-1),0.25,colors.grey),
        ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
        ("FONTSIZE",(0,0),(-1,-1),9),
        ("ALIGN",(1,1),(-1,-1),"RIGHT"),
    ]))
    story.append(t_cash)
    story.append(Spacer(1, 0.10 * inch))

    portfolio_rows = [
        ["Item", "Amount"],
        ["Portfolio Value @ prior close (pre-trade)", _fmt(pre_portfolio_value, "market_value")],
        ["Securities Value @ prior close", _fmt(pre_securities_value, "market_value")],
        ["Cash Value @ prior close", _fmt(pre_cash_value, "cash")],
        ["", ""],
        ["Portfolio Value @ execution-day close (post-trade)", _fmt(post_portfolio_value, "market_value")],
        ["Securities Value @ execution-day close", _fmt(post_securities_value, "market_value")],
        ["Cash Value @ execution-day close (audited)", _fmt(post_cash_value, "cash")],
        ["Weekly Portfolio Return", _fmt_pct(weekly_portfolio_return)],
    ]


    t_port = Table(portfolio_rows, hAlign="LEFT")
    t_port.setStyle(TableStyle([
        ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
        ("GRID",(0,0),(-1,-1),0.25,colors.grey),
        ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
        ("FONTSIZE",(0,0),(-1,-1),9),
        ("ALIGN",(1,1),(-1,-1),"RIGHT"),
    ]))
    story.append(t_port)
    story.append(Spacer(1, 0.15 * inch))

    summary = [
        ["Item", "Value"],
        ["Trades (count)", _fmt(wk["trades"])],
        ["Buys / Sells", f"{int(wk['buys'])} / {int(wk['sells'])}"],
        ["Gross notional ($)", _fmt(wk["gross_notional"], "gross_notional")],
        ["Broker fees ($)", _fmt(wk["fees"], "fees")],

        ["", ""],
        [f"--- SLIPPAGE — {SIG_LABEL} — THIS EXECUTION DAY ---", ""],
        ["SigFill slip (net $)", _fmt(day_sig["net_dollars"], "sig_slip_net_dollars")],
        ["SigFill slip (gross cost $)", _fmt(day_sig["gross_cost_dollars"], "sig_slip_gross_cost_dollars")],
        ["SigFill slip (gross improve $)", _fmt(day_sig["gross_improve_dollars"], "sig_slip_gross_improve_dollars")],
        ["SigFill slip (net bps)", _fmt(day_sig["net_bps"], "sig_slip_net_bps")],
        ["SigFill slip per trade ($)", _fmt(day_sig_per_trade, "sig_slip_per_trade_dollars")],

        ["", ""],
        [f"--- SLIPPAGE — {ORD_LABEL} — THIS EXECUTION DAY ---", ""],
        ["OrdFill slip (net $)", _fmt(day_ord["net_dollars"], "ord_slip_net_dollars")],
        ["OrdFill slip (gross cost $)", _fmt(day_ord["gross_cost_dollars"], "ord_slip_gross_cost_dollars")],
        ["OrdFill slip (gross improve $)", _fmt(day_ord["gross_improve_dollars"], "ord_slip_gross_improve_dollars")],
        ["OrdFill slip (net bps)", _fmt(day_ord["net_bps"], "ord_slip_net_bps")],
        ["OrdFill slip per trade ($)", _fmt(day_ord_per_trade, "ord_slip_per_trade_dollars")],

        ["Missing signal_price rows", str(int(wk.get("missing_signal_price", 0)))],
        ["Missing order_price rows", str(int(wk.get("missing_order_price", 0)))],

        ["", ""],
        ["--- YTD (through this execution day) ---", ""],
        ["YTD trades", f"{ytd_trades:,}"],
        ["YTD fees ($)", "N/A" if np.isnan(ytd_fees) else f"{ytd_fees:,.2f}"],

        ["YTD SigFill slip (net $)", _fmt(ytd_sig_day["net_dollars"], "ytd_sig_slip_net_dollars")],
        ["YTD SigFill slip (gross cost $)", _fmt(ytd_sig_day["gross_cost_dollars"], "ytd_sig_slip_gross_cost_dollars")],
        ["YTD SigFill slip (gross improve $)", _fmt(ytd_sig_day["gross_improve_dollars"], "ytd_sig_slip_gross_improve_dollars")],
        ["YTD SigFill slip (net bps)", _fmt(ytd_sig_day["net_bps"], "ytd_sig_slip_net_bps")],
        ["YTD SigFill slip per trade ($)", _fmt(ytd_sig_per_trade, "ytd_sig_slip_per_trade_dollars")],

        ["", ""],
        ["YTD OrdFill slip (net $)", _fmt(ytd_ord["net_dollars"], "ytd_ord_slip_net_dollars")],
        ["YTD OrdFill slip (gross cost $)", _fmt(ytd_ord["gross_cost_dollars"], "ytd_ord_slip_gross_cost_dollars")],
        ["YTD OrdFill slip (gross improve $)", _fmt(ytd_ord["gross_improve_dollars"], "ytd_ord_slip_gross_improve_dollars")],
        ["YTD OrdFill slip (net bps)", _fmt(ytd_ord["net_bps"], "ytd_ord_slip_net_bps")],
        ["YTD OrdFill slip per trade ($)", _fmt(ytd_ord_per_trade, "ytd_ord_slip_per_trade_dollars")],
    ]

    # ============================================================
    # STEP 6: Add ranked stocks table display (around line 1900)
    # ============================================================
    # ADD this section AFTER the summary table and BEFORE the trades tables:

    # -----------------------------
    # NEW: RANKED STOCKS TABLE (BEFORE FILTERS)
    # -----------------------------
    if not day_rankings.empty:
        story.append(Paragraph("All Ranked Stocks (Top Percentile) — Pre-Filter", styles["Header"]))
        
        # Sort by slope rank
        day_rankings_display = day_rankings.sort_values("slope_rank", na_position="last")
        
        # Select columns to display
        rank_cols = [
            "slope_rank",
            "ticker",
            "traded_flag",
            "no_trade_reason",
            "slope_adj",
            "target_weight",
            "current_weight",
            "target_shares",
            "current_shares",
            "close_adj",
        ]
        rank_cols = [c for c in rank_cols if c in day_rankings_display.columns]
        
        make_table(
            story,
            "Ranked Stocks (shows which traded and reasons for not trading)",
            day_rankings_display,
            cols=rank_cols,
            font_size=6
        )
    tbl = Table(summary, hAlign="LEFT")
    tbl.setStyle(TableStyle([
        ("BACKGROUND",(0,0),(-1,0),colors.lightgrey),
        ("GRID",(0,0),(-1,-1),0.25,colors.grey),
        ("FONTNAME",(0,0),(-1,-1),"Helvetica"),
        ("FONTSIZE",(0,0),(-1,-1),9),
        ("ALIGN",(1,1),(-1,-1),"RIGHT"),
    ]))
    story.append(tbl)
    story.append(Spacer(1, 0.15 * inch))

    # Sort trades for the table: BUYS then SELLS, each by best slope rank -> worst
    # ------------------------------------------------------------
    # Executed Trades tables: SELLS first, then BUYS (each w/ totals)
    # ------------------------------------------------------------

    # Identify rank column (same logic as sort_trades_by_rank, but we split tables)
    # ------------------------------------------------------------
    # Executed Trades tables: SELLS first, then BUYS (each w/ totals)
    # FIX: cash_before/cash_after shown in the tables must be sequential
    #      in DISPLAY ORDER (SELLS table order, then BUYS table order).
    # ------------------------------------------------------------

    rank_col_used = next((c for c in RANK_COL_CANDIDATES if c in day.columns), None)
    sum_cols = ["gross_notional", "broker_fee", SIG_SLIP_COL, ORD_SLIP_COL, "net_cash_impact"]

    # We will show recomputed sequential cash in these SAME column names:
    #   cash_before, cash_after
    # The audited originals remain available as:
    #   cash_before_audited, cash_after_audited
    base_cols = [
        "signal_date","exec_date","ticker","side","shares",
        "signal_price","order_price","fill_price",
        "gross_notional","broker_fee",
        rank_col_used,
        SIG_SLIP_COL, ORD_SLIP_COL,
        "net_cash_impact",
        "cash_before","cash_after",            # <-- sequential in display order
        "pos_before","pos_after",
        "broker_order_id"
    ]
    base_cols = [c for c in base_cols if c]  # drop None if no rank col

    # starting cash for DISPLAY sequencing = audited cash BEFORE first trade (ledger order)
    start_cash_display = float(cash_before_first) if pd.notna(cash_before_first) else 0.0

    # ---- SELLS table (first) ----
    day_sells = day[_norm_side(day["side"]) == "SELL"].copy()
    if rank_col_used is not None and rank_col_used in day_sells.columns:
        day_sells[rank_col_used] = pd.to_numeric(day_sells[rank_col_used], errors="coerce")
        day_sells = day_sells.sort_values(rank_col_used, ascending=True, na_position="last", kind="mergesort")

    day_sells_disp, end_cash_after_sells = add_sequential_display_cash(day_sells, start_cash_display)

    # rename display cash into the standard names the report already uses
    if day_sells_disp is not None and not day_sells_disp.empty:
        day_sells_disp = day_sells_disp.rename(columns={
            "cash_before_display": "cash_before",
            "cash_after_display": "cash_after",
            "cash_delta_display": "cash_delta",
        })

    day_sells_tbl = append_totals_row(day_sells_disp, sum_cols=sum_cols, label_col="ticker", label="TOTAL")

    # put start/end cash on totals row (so totals row is meaningful, not summed)
    if day_sells_tbl is not None and not day_sells_tbl.empty:
        last_idx = day_sells_tbl.index[-1]
        if "cash_before" in day_sells_tbl.columns:
            day_sells_tbl.loc[last_idx, "cash_before"] = start_cash_display
        if "cash_after" in day_sells_tbl.columns:
            day_sells_tbl.loc[last_idx, "cash_after"] = end_cash_after_sells

    sell_cols = [c for c in base_cols if c in day_sells_tbl.columns]

    make_table(
        story,
        "Executed SELLS (Sorted by slope rank best→worst) — cash sequential in display order",
        day_sells_tbl,
        cols=sell_cols,
        font_size=6.5,
        bold_last_row=True
    )

    # ---- BUYS table (second) ----
    day_buys = day[_norm_side(day["side"]) == "BUY"].copy()
    if rank_col_used is not None and rank_col_used in day_buys.columns:
        day_buys[rank_col_used] = pd.to_numeric(day_buys[rank_col_used], errors="coerce")
        day_buys = day_buys.sort_values(rank_col_used, ascending=True, na_position="last", kind="mergesort")

    # BUYS start where SELLS ended (so cash is sequential across both tables)
    day_buys_disp, end_cash_after_buys = add_sequential_display_cash(day_buys, end_cash_after_sells)

    if day_buys_disp is not None and not day_buys_disp.empty:
        day_buys_disp = day_buys_disp.rename(columns={
            "cash_before_display": "cash_before",
            "cash_after_display": "cash_after",
            "cash_delta_display": "cash_delta",
        })

    day_buys_tbl = append_totals_row(day_buys_disp, sum_cols=sum_cols, label_col="ticker", label="TOTAL")

    if day_buys_tbl is not None and not day_buys_tbl.empty:
        last_idx = day_buys_tbl.index[-1]
        if "cash_before" in day_buys_tbl.columns:
            day_buys_tbl.loc[last_idx, "cash_before"] = end_cash_after_sells
        if "cash_after" in day_buys_tbl.columns:
            day_buys_tbl.loc[last_idx, "cash_after"] = end_cash_after_buys

    buy_cols = [c for c in base_cols if c in day_buys_tbl.columns]

    make_table(
        story,
        "Executed BUYS (Sorted by slope rank best→worst) — cash sequential in display order",
        day_buys_tbl,
        cols=buy_cols,
        font_size=6.5,
        bold_last_row=True
    )



    # --- Portfolio Holdings as-of this execution day (NOT global asof) ---
    exec_slice = exec_df[exec_df["exec_date"].dt.normalize() <= exec_dt].copy()
    port_close_df = build_portfolio_snapshot_from_ledger(exec_slice, px_by_ticker, exec_dt)

    story.append(Paragraph("Portfolio Holdings (as of execution-day close)", styles["Header"]))
    story.append(Paragraph(
        "Pricing rule: use the last available close on or before the execution day. "
        "See price_date_used per ticker (will never show a future date like Dec 24 for Dec 18).",
        styles["Small"]
    ))

    make_table(
        story,
        "Holdings at Close",
        port_close_df,
        cols=["ticker","shares","avg_cost","current_price","price_date_used","position_return_pct","first_acquired","market_value","weight_pct"],
        font_size=8
    )

    story.append(PageBreak())


doc.build(story)

print("=== COMPLETE ===")
print(f"Reconciliation report saved → {pdf_path}")


C:\Users\farty\AppData\Local\Temp\ipykernel_14660\470971253.py:1203: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slip_by_day = weekly_grp.apply(_slip_apply).reset_index().rename(columns={"exec_date_only": "exec_date"})
C:\Users\farty\AppData\Local\Temp\ipykernel_14660\470971253.py:215: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([out, pd.DataFrame([totals])], ignore_index=True)
C:\Users\farty\AppData\Local\Temp\ipykernel_14660\47097125

=== COMPLETE ===
Reconciliation report saved → ./29a-2G_reconciliation_reports\reconciliation_report_20251230-121713.pdf
